In [1]:
#this is the same code we used for a CNN predicting a number based on a handmade drawing of it
#for this, i forked another repository: https://github.com/JosePabloGFnl/exportacion-numeros

import tensorflow as tf
import tensorflow_datasets as tfds

#MNIST data set
datos, metadatos = tfds.load('emnist', as_supervised=True, with_info=True)

datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

nombres_clases = metadatos.features['label'].names

#normalizing function, makes it faster
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #0-255 to 0-1
  return imagenes, etiquetas

datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

#creating the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32,
                kernel_size=(5, 5),
                strides=(2, 2),
                input_shape=(28, 28, 1),
                activation='relu'))
model.add(tf.keras.layers.Conv2D(64,
                kernel_size=(3, 3),
                activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(65, activation=tf.nn.softmax))

#Compiling the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

TAMANO_LOTE = 32

#shuffling
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

#training
historial = model.fit(datos_entrenamiento, epochs=5, steps_per_epoch= math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/697932 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/emnist/byclass/3.0.0.incompleteKNDC1I/emnist-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/116323 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/emnist/byclass/3.0.0.incompleteKNDC1I/emnist-test.tfrecord*...:   0%|         …

Dataset emnist downloaded and prepared to ~/tensorflow_datasets/emnist/byclass/3.0.0. Subsequent calls will reuse this data.
Epoch 1/5
21811/21811 [==============================] - 508s 21ms/step - loss: 0.5555 - accuracy: 0.8143
Epoch 2/5
21811/21811 [==============================] - 467s 21ms/step - loss: 0.4450 - accuracy: 0.8427
Epoch 3/5
21811/21811 [==============================] - 474s 22ms/step - loss: 0.4277 - accuracy: 0.8480
Epoch 4/5
21811/21811 [==============================] - 481s 22ms/step - loss: 0.4162 - accuracy: 0.8508
Epoch 5/5
21811/21811 [==============================] - 475s 22ms/step - loss: 0.4117 - accuracy: 0.8527


In [3]:
#exporting the model to h5
model.save('numeros.h5')

In [4]:
!ls

'~'   numeros.h5   sample_data


In [5]:
!pip install tensorflow js

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 8.0 MB/s 
  Created wheel for js: filename=js-1.0-py3-none-any.whl size=2907 sha256=a5f80aadc47a962ca2dde62ff6014f3652612e620b3a0346732d3b6cf4c104fb
  Stored in directory: /root/.cache/pip/wheels/bf/92/56/6597c8f32980ee64af7e864796e8d7423eb67eafff5415a4ae
  Created wheel for shutilwhich: filename=shutilwhich-1.1.0-py3-none-any.whl size=2781 sha256=995b2dca6593a32bd8fe30fa53a80b59e9830e83c9f1dcaadaca05816b05f8be
  Stored in directory: /root/.cache/pip/wheels/4c/c0/ce/5a47f35186acbe73e38811df30daaf798c81a7599bb2d5995a
Successfully built js shutilwhich


In [6]:
#folder with output files
!mkdir carpeta_salida

In [8]:
#export to output folder
!tensorflowjs_converter --input_format keras numeros.h5 carpeta_salida

/bin/bash: tensorflowjs_converter: command not found


In [9]:
!ls carpeta_salida

In [10]:
#note, if tensorflow does not work in the page, do the following:
#open cmd > cd [folder] > python -m http.server 8000
#in the address, localhost:8000/index.html